# Group 13 Project Proposal

In [1]:
pip install -U altair

Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.



**Title: Credit Score Classification**

https://www.kaggle.com/datasets/parisrohan/credit-score-classification/data 


**Introduction:**
A credit report is a summary of a person’s credit history and is created when you borrow money or apply for a credit card. A credit score is a 3 digit number calculated based on your credit report that summarizes how well you manage your credit and how risky it would be for someone to lend you money. The higher credit score the better your rating is.


A credit score is calculated based on a few different factors such as:
- a person's anual income
- the number of credit cards they have
- the number of loans they have
- their credit card payment history
- how old their credit is
and more




**The Question we aim to answer:** Can we classify someone’s credit score based on certain banking history and financial traits (shown above)?

**Dataset description:** The dataset contains the bank and credit-related information of many individuals that have been amassed by a global finance company. It contains 27 columns of these information such as bank account history, loans, debt and EMI along with the number of credit cards a person has and their credit card payment history


In [2]:
import random
import altair as alt
import pandas as pd
import numpy as np
from sklearn import set_config
from sklearn.compose import make_column_transformer
from sklearn.metrics.pairwise import euclidean_distances
from sklearn.neighbors import KNeighborsClassifier
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.metrics import confusion_matrix
from sklearn.model_selection import GridSearchCV, cross_validate

np.random.seed(999) # setting the seed

### Reading the data from a URL

In [3]:
url_credit = "https://raw.githubusercontent.com/vedika37/dsci100-proj/main/train.csv"
credit = pd.read_csv(url_credit,sep = ",", low_memory=False)
credit.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 28 columns):
 #   Column                    Non-Null Count   Dtype  
---  ------                    --------------   -----  
 0   ID                        100000 non-null  object 
 1   Customer_ID               100000 non-null  object 
 2   Month                     100000 non-null  object 
 3   Name                      90015 non-null   object 
 4   Age                       100000 non-null  object 
 5   SSN                       100000 non-null  object 
 6   Occupation                100000 non-null  object 
 7   Annual_Income             100000 non-null  object 
 8   Monthly_Inhand_Salary     84998 non-null   float64
 9   Num_Bank_Accounts         100000 non-null  int64  
 10  Num_Credit_Card           100000 non-null  int64  
 11  Interest_Rate             100000 non-null  int64  
 12  Num_of_Loan               100000 non-null  object 
 13  Type_of_Loan              88592 non-null   ob

As we can see from the output of credit.info() the number of non-null observations are not the same across all variables, so to be able to analyze patterns in our data, we will first have to do some data cleaning. 

### Cleaning Data 

In [4]:
# dropping null values and columns not used in analysis

# Predictors:

# - number of delayed payments 
# - delay from due date 

credit = credit[['Num_of_Delayed_Payment', 'Delay_from_due_date', 'Credit_Score']]
credit = credit.dropna()
credit

,Num_of_Delayed_Payment,Delay_from_due_date,Credit_Score
0,7,3,Good
2,7,3,Good
3,4,5,Good
5,4,8,Good
6,8_,3,Good
...,...,...,...
99994,6,20,Poor
99995,7,23,Poor
99996,7,18,Poor
99997,6,27,Poor


In [5]:
credit["Credit_Score"].unique()

array(['Good', 'Standard', 'Poor'], dtype=object)

We see here that our output variable i.e the credit score takes three unique values -> Good, Standard and Poor.

In [6]:
credit = credit[credit['Num_of_Delayed_Payment'].str.isdigit()]
credit

,Num_of_Delayed_Payment,Delay_from_due_date,Credit_Score
0,7,3,Good
2,7,3,Good
3,4,5,Good
5,4,8,Good
7,6,3,Standard
...,...,...,...
99994,6,20,Poor
99995,7,23,Poor
99996,7,18,Poor
99997,6,27,Poor


In [7]:
credit= credit.astype({'Num_of_Delayed_Payment': int})
credit


,Num_of_Delayed_Payment,Delay_from_due_date,Credit_Score
0,7,3,Good
2,7,3,Good
3,4,5,Good
5,4,8,Good
7,6,3,Standard
...,...,...,...
99994,6,20,Poor
99995,7,23,Poor
99996,7,18,Poor
99997,6,27,Poor


### Summarizing Data

In [8]:
# each column along with its datatype
credit.dtypes

Num_of_Delayed_Payment     int32
Delay_from_due_date        int64
Credit_Score              object
dtype: object

In [9]:
credit.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 89632 entries, 0 to 99999
Data columns (total 3 columns):
 #   Column                  Non-Null Count  Dtype 
---  ------                  --------------  ----- 
 0   Num_of_Delayed_Payment  89632 non-null  int32 
 1   Delay_from_due_date     89632 non-null  int64 
 2   Credit_Score            89632 non-null  object
dtypes: int32(1), int64(1), object(1)
memory usage: 2.4+ MB


In [10]:
# generating descriptive statistics for numeric and 'object' type columns
credit.describe()

,Num_of_Delayed_Payment,Delay_from_due_date
count,89632.000000,89632.000000
mean,31.259863,21.175328
std,227.725486,14.864670
min,0.000000,-5.000000
25%,9.000000,10.000000
50%,14.000000,18.000000
75%,18.000000,28.000000
max,4397.000000,67.000000


### Taking a Random Sample
Since our current dataset has 81549 values, it is too  large to perform 

In [11]:
credit['Credit_Score'].value_counts(normalize=True)

Standard    0.532366
Poor        0.291693
Good        0.175942
Name: Credit_Score, dtype: float64

In [12]:
credit_sample = credit.sample(n = 1000)
credit_sample['Credit_Score'].value_counts(normalize=True)

Standard    0.545
Poor        0.280
Good        0.175
Name: Credit_Score, dtype: float64

#### Impact Of Taking Subset of Data

We have limited our sample size to 1,000 observations, despite the original dataset containing over 80,000 entries. Training our model on the full dataset might yield more representative results, but this approach demands significant computational resources and time. To mitigate this, we've ensured that the proportions of output categories in our sample closely mirror those in the original dataset, thus minimizing the risk of bias in our results. However, it's notable that the 'standard' category constitutes approximately 53.3% of our sample outputs. This could suggest a sampling bias, a factor often beyond the control of data scientists and a potential limitation of our study. To further address this, we could consider selecting a dataset where all outputs are equally represented. However, this approach may not accurately reflect the actual population demographics, presenting a trade-off between balanced representation and real-world application.

### Train - Test Split 

In [13]:
from sklearn.model_selection import train_test_split

train, test = train_test_split(
    credit_sample, train_size=0.75, stratify=credit_sample["Credit_Score"]
)
print(train.info())
print(test.info())

<class 'pandas.core.frame.DataFrame'>
Int64Index: 750 entries, 25469 to 14556
Data columns (total 3 columns):
 #   Column                  Non-Null Count  Dtype 
---  ------                  --------------  ----- 
 0   Num_of_Delayed_Payment  750 non-null    int32 
 1   Delay_from_due_date     750 non-null    int64 
 2   Credit_Score            750 non-null    object
dtypes: int32(1), int64(1), object(1)
memory usage: 20.5+ KB
None
<class 'pandas.core.frame.DataFrame'>
Int64Index: 250 entries, 28983 to 95411
Data columns (total 3 columns):
 #   Column                  Non-Null Count  Dtype 
---  ------                  --------------  ----- 
 0   Num_of_Delayed_Payment  250 non-null    int32 
 1   Delay_from_due_date     250 non-null    int64 
 2   Credit_Score            250 non-null    object
dtypes: int32(1), int64(1), object(1)
memory usage: 6.8+ KB
None


In [14]:
print("Value counts for training data\n",train["Credit_Score"].value_counts(normalize=True))
print("\nValue counts for testing data\n",test["Credit_Score"].value_counts(normalize=True))

Value counts for training data
 Standard    0.545333
Poor        0.280000
Good        0.174667
Name: Credit_Score, dtype: float64

Value counts for testing data
 Standard    0.544
Poor        0.280
Good        0.176
Name: Credit_Score, dtype: float64


### Visualizing Data - Distribution of Predictor Variables

In [15]:
# initializing list of predictors
predictors = ["Delay_from_due_date", "Num_of_Delayed_Payment"]

# visualizing training data

# removing outliers - just for the visualization
def remove_outliers(column):
    Q1 = column.quantile(0.25)
    Q3 = column.quantile(0.75)
    IQR = Q3 - Q1
    return (column >= Q1 - 1.5 * IQR) & (column <= Q3 + 1.5 * IQR)

# Apply outlier removal to each column
for col in predictors:
    train = train[remove_outliers(train[col])]

train_pairplot = alt.Chart(train).mark_point(opacity=0.4).encode(
    alt.X(alt.repeat("row"), type="quantitative"),
    alt.Y(alt.repeat("column"), type="quantitative"),
    color = alt.Color("Credit_Score").title("Credit_Score")
).properties(
    width=200,
    height=200
).repeat(
    column=predictors,
    row=predictors
)

train_pairplot

alt.RepeatChart(...)

Delay from due date vs Num of delayed payments. We would expect to see a linaer relation between these two variables (generally speaking). Our expectations are met as seen in the visualization above. The two variables show a (weak) positive relation.

### Methods

We plan to conduct our data analysis using the K Nearest Neighbors Classification algrorithm. We'll choose the best value of k using cross-validation and then use the following predictors to predict whether someone's Credit Score is Good, Standard or Poor.

Predictors:

- number of delayed payments 
- delay from due date 
**Describing our visualizations**
We will plot histograms to visualize the distributions of our predictors. This will help us explore the relation between predicted credit score and the factors that may impact the credit score more than others like income, missed payments, credit utilization ratios.


In [16]:
# training_labels = subset_train["Credit_Score"]
# training_labels

In [17]:
knn = KNeighborsClassifier(n_neighbors=57)
knn

KNeighborsClassifier(n_neighbors=57)

In [18]:
preprocessor = make_column_transformer(
    (StandardScaler(), predictors),
    remainder='passthrough',
    verbose_feature_names_out=False
)
preprocessor

ColumnTransformer(remainder='passthrough',
                  transformers=[('standardscaler', StandardScaler(),
                                 ['Delay_from_due_date',
                                  'Num_of_Delayed_Payment'])],
                  verbose_feature_names_out=False)

In [19]:
X = train.drop(columns='Credit_Score')
y = train["Credit_Score"]

train_fit = make_pipeline(preprocessor, knn).fit(X, y)

train_fit

Pipeline(steps=[('columntransformer',
                 ColumnTransformer(remainder='passthrough',
                                   transformers=[('standardscaler',
                                                  StandardScaler(),
                                                  ['Delay_from_due_date',
                                                   'Num_of_Delayed_Payment'])],
                                   verbose_feature_names_out=False)),
                ('kneighborsclassifier', KNeighborsClassifier(n_neighbors=57))])

In [20]:
test_predictions = test.assign(
    predicted = train_fit.predict(test[predictors])
)
test_predictions

,Num_of_Delayed_Payment,Delay_from_due_date,Credit_Score,predicted
28983,23,47,Standard,Poor
1431,20,25,Standard,Standard
27040,5,18,Standard,Standard
9783,1,11,Standard,Good
29776,7,15,Standard,Standard
...,...,...,...,...
69945,6,16,Standard,Standard
60433,14,11,Standard,Standard
88064,17,5,Standard,Standard
50620,10,10,Good,Standard


In [21]:
correct_preds = test_predictions[
    test_predictions['Credit_Score'] == test_predictions['predicted']
]

correct_preds.shape[0] / test_predictions.shape[0]

0.62

In [22]:
X_test = test[predictors]
y_test = test["Credit_Score"]
accuracy = train_fit.score(X_test,y_test)
accuracy

0.62

### Choosing the best K

In [23]:
param_grid = {
    "kneighborsclassifier__n_neighbors": range(1, 100, 5),
}
tune_pipe = make_pipeline(preprocessor, KNeighborsClassifier())

In [24]:
knn_tune_grid = GridSearchCV(
    estimator = tune_pipe, param_grid = param_grid, cv=25,
)
knn_tune_grid

GridSearchCV(cv=25,
             estimator=Pipeline(steps=[('columntransformer',
                                        ColumnTransformer(remainder='passthrough',
                                                          transformers=[('standardscaler',
                                                                         StandardScaler(),
                                                                         ['Delay_from_due_date',
                                                                          'Num_of_Delayed_Payment'])],
                                                          verbose_feature_names_out=False)),
                                       ('kneighborsclassifier',
                                        KNeighborsClassifier())]),
             param_grid={'kneighborsclassifier__n_neighbors': range(1, 100, 5)})

In [25]:
knn_model_grid = knn_tune_grid.fit(X, y)

accuracies_grid = pd.DataFrame(knn_model_grid.cv_results_)
#accuracies_grid

In [26]:
accuracy_versus_k_grid = alt.Chart(accuracies_grid).mark_line(point=True).encode(
    x=alt.X("param_kneighborsclassifier__n_neighbors")
        .title("neighbors")
        .scale(zero=False),
    y=alt.Y("mean_test_score")
        .title("Mean test score")
        .scale(zero=False)
)
accuracy_versus_k_grid

alt.Chart(...)

### Expected outcomes and significance
- *What do you expect to find?* 
  
  Find if number of late payments and days from late payment interact with each other to have an impact on someone's credit score category.
  
- *What impact could such findings have?*
  - Helping banks predict whether it is a good idea to issue a new credit card to a new user
  - Can influence an individual’s credit limit/interest rate.
  - Studying the relation of individual factors with credit score category classification.
  
- *What future questions could this lead to?* 
  
  How do we evaluate or categorize a new person who has just started working and does not have a long enough credit history. 


### Discussion


In our credit score category classification project, we discovered that a k-nearest neighbors (KNN) model with a k-value of 57 yields the highest accuracy of 62%. This finding was somewhat anticipated given the nature of our dataset and the typical behavior of KNN models. A moderately high k-value like 57 helps in achieving a balance between overfitting and bias. It's larger than the very small k-values which tend to overfit, but not so large as to introduce significant bias, which can be a problem with very high k-values. This middle ground suggests that our model is less prone to overfitting and is likely to have a smoother decision boundary, making it suitable for larger datasets.

What we found: our predictors (num of delayed payments and days from delayed payments) show a weak positive relation as we would expect. But it is unlikely and hard to find a strong positive relation owing to the difference in behavioural patterns of consumers, however, that discussion is outside the scope of this project. 

The impact of these findings is notable in the field of credit scoring. By efficiently predicting creditworthiness with a balanced bias-variance tradeoff, financial institutions can make more informed decisions, potentially reducing the risk of default while offering credit opportunities to eligible individuals.

Potential Shortcomings:investigating the impact of feature selection and data quality on the model's performance could lead to more robust and accurate predictions. Lastly, considering ethical implications and the model's behavior in diverse, real-world scenarios would be essential for responsible application. Since we see that existing research cited discriminates based on credit history age. Additionally, we only used two predictor variables but using multiple variables and studying their interactions is something worth looking into for improving the accuracy of the model. With adding each new predictor, the interaction possibilities grows multiplicatively. So, the process can be time consuming but might yield better results. 

### References


- Debt.org. (2023, September 1). Credit Scoring Models: FICO, VantageScore & More. Retrieved from https://www.debt.org/credit/report/scoring-models/                                     
- Akin, J. (2023, July 29). What Affects Your Credit Scores? Experian. https://www.experian.com/blogs/ask-experian/credit-education/score-basics/what-affects-your-credit-scores/